In [17]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

In [18]:
#Load Dataset

In [19]:
df = pd.read_csv ('modcloth (prepro).csv')
df=df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [20]:
#findout avarage rateing and no of rateing for each brand

In [21]:
Avarage_ratings = pd.DataFrame(df.groupby('brand')['rating'].mean())
Avarage_ratings ['num of ratings'] = pd.DataFrame(df.groupby('brand')['rating'].count())

In [22]:
Avarage_ratings.shape

(31, 2)

In [23]:
#charecter encodeing

In [24]:
le = preprocessing.LabelEncoder()
le.fit_transform(list(df['brand']))
df['labelencoded_Brand'] = le.fit_transform(list(df['brand']))

In [25]:
df

,item_id,user_id,rating,size,fit,user_attr,model_attr,category,brand,year,labelencoded_Brand
328,21296,Nikki,5,3.0,Just right,Small,Small,Bottoms,ModCloth,2013,19
329,21296,ellen,5,1.0,Just right,Large,Small,Bottoms,ModCloth,2013,19
340,21296,Sarita,5,2.0,Just right,Small,Small,Bottoms,ModCloth,2013,19
351,21296,kacikorn,4,1.0,Just right,Small,Small,Bottoms,ModCloth,2013,19
361,21296,lesleydawn88,4,1.0,Just right,Small,Small,Bottoms,ModCloth,2013,19
...,...,...,...,...,...,...,...,...,...,...,...
99780,147244,Stacie,5,5.0,Just right,Small,Small&Large,Dresses,Chi Chi London,2016,4
99787,6454,Evil,2,3.0,Slightly large,Small,Small&Large,Dresses,ModCloth,2017,19
99789,70538,KaliK,5,6.0,Just right,Large,Small&Large,Dresses,ModCloth,2017,19
99798,119314,Doofus,1,2.0,Just right,Small,Small&Large,Dresses,Retrolicious,2017,23


Prepare pivot table brand vs user_id

In [27]:
BRAND_NAME = df.pivot_table(index=['brand'],columns='user_id',values='rating').fillna(0)
BRAND_NAME.head()

user_id,"""Ferrari"")",#NAME?,'Chelle,00erin,00izzyb,02draper,044de0c8,10097685jk,1234babysitter,16lexwei,...,zoefaye,zoehelen,zoekerchen,zoevkay,zoey,zombie66_79,zombiscuit,zoshia,zpsarouthakis,zuzu_zoom
brand,,,,,,,,,,,,,,,,,,,,,
Alice's Pig,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BB Dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Banned,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blue Platypus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chi Chi London,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [28]:
#user_input

In [29]:
User_input=le.transform(["Alice's Pig"])


In [30]:
#fire user input Query to get the result

In [31]:
query_point=BRAND_NAME.iloc[User_input,:].values.reshape(1, -1)

In [32]:
#prepare unsuperviced learning  model

In [33]:
BRAND_NAME_matrix = csr_matrix(BRAND_NAME.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(BRAND_NAME_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

Findout cosine distance of other brand and indices with respect to the quary point , Minimum distance have maximum likelyhood for similar brand recomendation

In [34]:
distances, indices = model_knn.kneighbors(query_point, n_neighbors = 6)

In [35]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(BRAND_NAME.index[User_input]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, BRAND_NAME.index[indices.flatten()[i]], distances.flatten()[i]))
        

Recommendations for Index(['Alice's Pig'], dtype='object', name='brand'):

1: Louche, with distance of 0.8207006996596798:
2: Sugarhill Boutique, with distance of 0.8929087914097977:
3: Daisey Natives, with distance of 0.9258549507046024:
4: Collectif, with distance of 0.9299403486654269:
5: Hell Bunny, with distance of 0.9510974306659346:


In [36]:
l2=[]
for i in range(0, len(distances.flatten())):
        l1=BRAND_NAME.index[indices.flatten()[i]].split(',')
        l2.append(l1)
        
        
        

In [37]:
l2 =pd.DataFrame(l2,columns=['brand'])

In [38]:
choice_of_brand=l2.merge(Avarage_ratings, how='inner', on='brand')

In [39]:
choice_of_brand

,brand,rating,num of ratings
0,Alice's Pig,3.500000,8
1,Louche,3.666667,3
2,Sugarhill Boutique,3.900000,20
3,Daisey Natives,4.857143,28
4,Collectif,3.850000,80
5,Hell Bunny,4.093023,86


Final result cointains of similar brand recomendation for particular brand(User input/quary point)

In [42]:
choice_of_brand.sort_values(by ='rating', ascending = 0)

,brand,rating,num of ratings
3,Daisey Natives,4.857143,28
5,Hell Bunny,4.093023,86
2,Sugarhill Boutique,3.900000,20
4,Collectif,3.850000,80
1,Louche,3.666667,3
0,Alice's Pig,3.500000,8
